## MPI parallel matrix-vector product in dolfinx/python

In this notebook we implement matrix-vector multiplication with MPI. 

1. For both matrices and vectors, we construct index maps and use that to distribute values. Basically, we create an IndexMap to describe the distribution scheme we want, and then we use the IndexMap to create objects (such as vectors and matrices) that have this distribution. 

2. After we define IndexMap and distributed objects we need, we show some basic arithmetic with those objects, such as adding vectors and matrix-vector product.

In [23]:
import logging
import ipyparallel as ipp

# create a cluster
cluster = ipp.Cluster(engines="mpi", n=2, log_level=logging.WARNING)
rc = cluster.start_and_connect_sync(activate=True)

  0%|          | 0/2 [00:00<?, ?engine/s]

In [24]:
%%px
# Find out rank, size
from mpi4py import MPI

comm = MPI.COMM_WORLD
world_rank = comm.Get_rank()
world_size = comm.Get_size()
print(f"I am rank {world_rank} / {world_size}")

[stdout:0] I am rank 0 / 2


[stdout:1] I am rank 1 / 2


We want to create an overlapping, ie ghosted, index map for our vectors and matrices. (see [dolfinx implementation](https://github.com/FEniCS/dolfinx/blob/160ed13eb476df99944072aec70bd46a6fcb9bcf/cpp/dolfinx/common/IndexMap.cpp))

What we need is:
- MPI communicator
- **source ranks** - ranks that own indices ghosted by the caller
- **destination ranks** - ranks that ghost indices owned by the caller
- local size of the index map (number of owned entries)
- global indices of ghost entries
- owner rank of each ghost entry on global communicator

In [25]:
def is_sorted(lst):
    return all(a <= b for a, b in zip(lst, lst[1:]))

In [26]:
class IndexMap():
    def __init__(comm, local_size, ghosts, owners):
        # fixme: perhaps hide these and use special methods to access
        # assume ghosts and owners are of type list
        assert len(ghosts) == len(owners)
        
        self.comm = comm
        # self.forward_comm = None
        # self.reverse_comm = None
        self.ghosts = ghosts
        self.owners = owners
        self.local_size = local_size
        
        # Get global size
        self.global_size = 0
        self.compute_size_global()
        
        # Get global offset (index) using global exclusive reduction
        self.local_range = None
        self.compute_local_range()
        
        # Get sources and destinations
        self.sources = None
        self.destinations = None
        self.compute_src_dest()
        
    def compute_src_dest():
        # src = sort(owners)
        # remove something unique from src
        # dest = sort(MPI.compute_graph_edges_nbx(self.comm, src))
        
        dest_ranks = np.unique(owners)
        mesh_to_data = self.comm.Create_dist_graph([self.comm.rank], [len(dest_ranks)], dest_ranks.tolist(), reorder=False)
        src, dest, _ = mesh_to_data.Get_dist_neighbors()
        # assume src and dest are lists
        # assert is_sorted(src)
        # assert is_sorted(dest)
        
        self.sources = src
        self.destinations = dest
        
    def local_to_global(local_ind):
        # todo
        global_ind = None
        return global_ind
    
    def compute_size_global():
        # todo: test
        self.comm.Iallreduce(self.local_size, self.global_size, op=MPI.SUM)
        
    def compute_local_range():
        # todo: test
        offset = 0
        self.comm.Iexscan(self.local_size, offset, op=MPI.SUM)
        self.local_range = range(offset, offset + self.local_size)


In [27]:
import dolfinx

In [28]:
# Stop cluster
cluster.stop_cluster_sync()